# Third Method Neural Network Implementations

<p><b>Author</b>: Jingze Dai</p>
<p><b>McMaster University</b>, Honors Computer Science (Coop) student</p>
<p><b>Personal Email Address</b>: <a>david1147062956@gmail.com</a>, or <a>dai.jingze@icloud.com</a></p>
<a href="https://github.com/daijingz">Github Homepage</a>
<a href="https://www.linkedin.com/in/jingze-dai/">Linkedin Webpage</a>
<a href="https://leetcode.com/david1147062956/">Leetcode Webpage</a>

<i>The original research's third featuring method with its corresponding ANN, CNN, and LSTM implementations.</i>

<i>Your Feedback is important for Jingze's further development. If you want to give feedback and suggestions, or you want to participate in working and learning together, please email Jingze at dai.jingze@icloud.com. If you want Jingze to provide contributions to your research or opensource project or you want Jingze to help you with any programming issues, please email Jingze at david1147062956@gmail.com. Thank you for your help.</i>

In [2]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install --upgrade tensorflow --user

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --user numpy==1.24.4

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade scipy

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import gdown

# Load the dataset
output_file = 'mixalldata_clean.csv'
df = pd.read_csv(output_file)

# Display the DataFrame
print(df.head())

   type      sendTime  sender  senderPseudo  messageID  class        posx  \
0     4  72002.302942  130137     101301377  422013806      0  266.982401   
1     4  72003.302942  130137     101301377  422023410      0  266.827208   
2     4  72004.302942  130137     101301377  422032081      0  266.420297   
3     4  72005.302942  130137     101301377  422040712      0  268.912026   
4     4  72006.302942  130137     101301377  422052949      0  268.242276   

        posy  posz    posx_n  ...  aclz    aclx_n    acly_n  aclz_n      hedx  \
0  32.336955   0.0  3.480882  ...   0.0  0.000862  0.000862     0.0 -0.102790   
1  34.624145   0.0  3.546261  ...   0.0  0.000107  0.001040     0.0 -0.099856   
2  38.836461   0.0  3.544045  ...   0.0  0.000172  0.001661     0.0 -0.099856   
3  45.414229   0.0  3.340080  ...   0.0  0.000171  0.001654     0.0 -0.100172   
4  53.729986   0.0  3.328872  ...   0.0  0.000193  0.001852     0.0 -0.097105   

       hedy  hedz     hedx_n     hedy_n  hedz_n  


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tffrom tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features]
Y = (df['class'] != 0).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, input_shape=(len(features),), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 74s 1ms/step - accuracy: 0.6902 - loss: 0.6245 - val_accuracy: 0.7628 - val_loss: 0.4963
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 15s 769us/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 15s 760us/step - accuracy: 0.7614 - loss: 0.4966
ANN Accuracy: 0.7618277668952942
ANN Precision: 0.9234229960308048
ANN Recall: 0.4504236618110434
ANN F1-score: 0.6054992599044486


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = (df['class'] != 0).astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred = model.predict(X_test)
Y_pred_binary = (Y_pred > 0.5).astype(int)

accuracy = accuracy_score(Y_test, Y_pred_binary)
precision = precision_score(Y_test, Y_pred_binary)
recall = recall_score(Y_test, Y_pred_binary)
f1 = f1_score(Y_test, Y_pred_binary)

print("CNN Accuracy:", accuracy)
print("CNN Precision:", precision)
print("CNN Recall:", recall)
print("CNN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 135s 2ms/step - accuracy: 0.7339 - loss: 0.5728 - val_accuracy: 0.7898 - val_loss: 0.4625
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 20s 998us/step
CNN Accuracy: 0.7885555009531083
CNN Precision: 0.9614509943815215
CNN Recall: 0.498941327563665
CNN F1-score: 0.6569572139151896


In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = (df['class'] != 0).astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(64, input_shape=(sequence_length, 1)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

print("LSTM Accuracy:", accuracy)
print("LSTM Precision:", precision)
print("LSTM Recall:", recall)
print("LSTM F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 286s 4ms/step - accuracy: 0.7590 - loss: 0.5038 - val_accuracy: 0.7914 - val_loss: 0.4638
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 38s 2ms/step - accuracy: 0.7902 - loss: 0.4651
LSTM Accuracy: 0.7911252975463867
LSTM Precision: 0.9606103203889181
LSTM Recall: 0.5060184274568336
LSTM F1-score: 0.662862397094003


In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def characterize_class(value):
    if value == 0:
        return 0
    elif value >= 1 and value <= 12:
        return 1
    else:
        return 2

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = df['class'].apply(characterize_class).astype(int).values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, input_shape=(len(features),), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 223s 3ms/step - accuracy: 0.3344 - loss: -4652460.0000 - val_accuracy: 0.3753 - val_loss: -68858608.0000
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 49s 2ms/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 45s 2ms/step - accuracy: 0.3735 - loss: -69851048.0000


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ANN Accuracy: 0.37397685647010803
ANN Precision: 0.43353787181448067
ANN Recall: 0.37397685621367155
ANN F1-score: 0.3640409172827655


In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

def characterize_class(value):
    if value == 0:
        return 0
    elif value >= 1 and value <= 12:
        return 1
    else:
        return 2

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = df['class'].apply(characterize_class).astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred = model.predict(X_test)
Y_pred_binary = (Y_pred > 0.5).astype(int)

accuracy = accuracy_score(Y_test, Y_pred_binary)
precision = precision_score(Y_test, Y_pred_binary, average='weighted')
recall = recall_score(Y_test, Y_pred_binary, average='weighted')
f1 = f1_score(Y_test, Y_pred_binary, average='weighted')

print("CNN Accuracy:", accuracy)
print("CNN Precision:", precision)
print("CNN Recall:", recall)
print("CNN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 295s 5ms/step - accuracy: 0.3406 - loss: -4908548.5000 - val_accuracy: 0.3646 - val_loss: -70431504.0000
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 23s 1ms/step


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CNN Accuracy: 0.3637462008695353
CNN Precision: 0.42212103826182124
CNN Recall: 0.3637462008695353
CNN F1-score: 0.354193808658255


In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

def characterize_class(value):
    if value == 0:
        return 0
    elif value >= 1 and value <= 12:
        return 1
    else:
        return 2

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = df['class'].apply(characterize_class).astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(64, input_shape=(sequence_length, 1)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

print("LSTM Accuracy:", accuracy)
print("LSTM Precision:", precision)
print("LSTM Recall:", recall)
print("LSTM F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 836s 13ms/step - accuracy: 0.4560 - loss: -70.9306 - val_accuracy: 0.3963 - val_loss: -326.7960
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.3959 - loss: -327.3655


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LSTM Accuracy: 0.3964727222919464
LSTM Precision: 0.46413267372450967
LSTM Recall: 0.3964727166873773
LSTM F1-score: 0.38950174668682536


In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features]
Y = df['class'].astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(64, input_shape=(len(features),), activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 79s 1ms/step - accuracy: 0.0138 - loss: -116202307584.0000 - val_accuracy: 0.0137 - val_loss: -1664064421888.0000
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 15s 753us/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 16s 812us/step - accuracy: 0.0136 - loss: -1679659106304.0000


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ANN Accuracy: 0.013424898497760296
ANN Precision: 0.00018022789993040557
ANN Recall: 0.013424898507266473
ANN F1-score: 0.0003556808209387275


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Assuming 'df' is your DataFrame
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features]
Y = df['class'].astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(128, input_shape=(len(features),), activation='relu'),  # Increased neurons
    Dropout(0.2),  # Dropout layer to prevent overfitting
    Dense(64, activation='relu'),  # Additional layer
    Dense(32, activation='relu'),  # Original layer
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)  # Increased epochs for more training

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)

Epoch 1/10


C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 84s 1ms/step - accuracy: 0.0139 - loss: -377139232768000.0000 - val_accuracy: 0.0137 - val_loss: -7907536766238720.0000
Epoch 2/10
 7271/63897 ━━━━━━━━━━━━━━━━━━━━ 1:03 1ms/step - accuracy: 0.0133 - loss: -8805897895673856.0000

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = df['class'].astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(sequence_length, 1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred = model.predict(X_test)
Y_pred_binary = (Y_pred > 0.5).astype(int)

accuracy = accuracy_score(Y_test, Y_pred_binary)
precision = precision_score(Y_test, Y_pred_binary, average='weighted')
recall = recall_score(Y_test, Y_pred_binary, average='weighted')
f1 = f1_score(Y_test, Y_pred_binary, average='weighted')

print("CNN Accuracy:", accuracy)
print("CNN Precision:", precision)
print("CNN Recall:", recall)
print("CNN F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 132s 2ms/step - accuracy: 0.0144 - loss: -190857625600.0000 - val_accuracy: 0.0137 - val_loss: -2633349726208.0000
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CNN Accuracy: 0.013424898507266473
CNN Precision: 0.00018022789993040557
CNN Recall: 0.013424898507266473
CNN F1-score: 0.0003556808209387275


In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import precision_score, recall_score, f1_score

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = df[features].values
Y = df['class'].astype(int).values

sequence_length = len(features)
X = X.reshape(-1, sequence_length, 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    LSTM(64, input_shape=(sequence_length, 1)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=1, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

print("LSTM Accuracy:", accuracy)
print("LSTM Precision:", precision)
print("LSTM Recall:", recall)
print("LSTM F1-score:", f1)

C:\Users\david\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


63897/63897 ━━━━━━━━━━━━━━━━━━━━ 295s 5ms/step - accuracy: 0.0141 - loss: -3793.4465 - val_accuracy: 0.0137 - val_loss: -15081.2393
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step
19968/19968 ━━━━━━━━━━━━━━━━━━━━ 41s 2ms/step - accuracy: 0.0136 - loss: -15181.0928


C:\Users\david\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


LSTM Accuracy: 0.013424898497760296
LSTM Precision: 0.00018022789993040557
LSTM Recall: 0.013424898507266473
LSTM F1-score: 0.0003556808209387275
